In [ ]:
using JuMP
using MadNLP
using ProgressMeter
using CairoMakie

In [ ]:
num_atoms = 15
num_bonds = num_atoms - 1
fps = 60
animation_fps = 60
animation_scale = 4
pref_dist = 1.
start_L = num_bonds*pref_dist
break_dist = 1.5*pref_dist
max_pull = 2*break_dist - pref_dist
max_push = max_pull
min_crack = 0.1*pref_dist
max_crack = start_L + max_pull - (num_bonds-1)*min_crack
pot_min = -1.
diss_coeff = 0.3
time_pull = 3.5.
l2_dissipation = false # if false => Kelvin-Voigt
soft_max_alpha = 5
time_horizon = 6*time_pull
animation_width = 800
hidpi_scaling = 2
fontsize = 16
only_video = true
experiments_folder = "experiments";

In [ ]:
function lennard_jones(dist_sq)
    # Lennard-jones of squared distances.
    # The factor cbrt(2) assures that the global minimum 
    # is attained at cur_dist == pref_dist
    q = pref_dist^2/(cbrt(2)*dist_sq)
    -4*pot_min*(q^6 - q^3)
#     q = pref_dist^2/(2*dist_sq)
#     -4*pot_min*(q^2 - q)
end

function dirichlet_delta(t)
    if t <= 2*time_pull
        return max_pull*sin(pi/2*t/time_pull)
    elseif t <= 4*time_pull
        return max_push*sin(pi/2*t/time_pull)
    elseif t <= 5*time_pull
        return max_pull*sin(pi/2*t/time_pull)
    else
        return max_pull
    end
end

In [ ]:
function compute_line_minmoves()
    step = 2

    minmove = Model(
        ()->MadNLP.Optimizer(
            print_level=MadNLP.WARN,
            blas_num_threads=2,
            # acceptable_tol=1e-8,
            # max_iter=1000
        )
    )

    prev_L = start_L
    L = start_L + dirichlet_delta((step-1)/fps)
    # uniform distribution of atoms at the start of the simulation
    @NLparameter(
        minmove, 
        prev_delta_y[i=1:num_bonds] == prev_L / num_bonds
    )
    @NLexpression(
        minmove, prev_y[i=1:num_atoms],
        sum(prev_delta_y[j] for j in 1:i-1),
    )

    @NLparameter(minmove, max_elong_sq[i=1:num_bonds] == value(prev_delta_y[i])^2)
    @NLparameter(minmove, broken[i=1:num_bonds] == 0.)

    delta_L = L - prev_L
    # initial guess: uniform expansion of all bonds
    @variable(
        minmove, min_crack <= delta_y[i=1:num_bonds] <= max_crack,
        start=value(prev_delta_y[i]) + delta_L/num_bonds
    )
    @expression(minmove, y[i=1:num_atoms], sum(delta_y[j] for j in 1:i-1))

    # Dirichlet boundary condition
    @constraint(minmove, dirichlet_right, y[end] == L)

    # Energy
    @expression(minmove, dist_sq[i=1:num_bonds], delta_y[i]^2)
    register(minmove, :W, 1, lennard_jones, autodiff = true)
    @NLexpression(
        minmove, energy_expr,
        sum((1-broken[i])*W(dist_sq[i]) + broken[i]*(
            (
                W(dist_sq[i])*exp(soft_max_alpha*W(dist_sq[i])) +
                W(max_elong_sq[i])*exp(soft_max_alpha*W(max_elong_sq[i]))
            ) / (exp(soft_max_alpha*W(dist_sq[i])) + exp(soft_max_alpha*W(max_elong_sq[i])))
        ) for i in 1:num_bonds)
    )

    # Dissipation
    if l2_dissipation
        @NLexpression(
            minmove, dissipation_expr,
            0.5*sum((y[i] - prev_y[i])^2 for i in 2:num_atoms-1)
        )
    else
        @NLexpression(
            minmove, dissipation_expr,
            0.5*sum((delta_y[i] - prev_delta_y[i])^2 for i in 1:num_bonds)
        )
    end

    @NLobjective(minmove, Min, energy_expr + diss_coeff*fps*dissipation_expr)

    optimize!(minmove)
    
    steps = [
        (value.(prev_y), value.(broken) .== 1.)
        (value.(y), value.(broken) .== 1.)
    ]
    
    @showprogress "Computing minmoves..." for step in 3:fps*time_horizon
        # update memory variable
        y_last = steps[end][1]
        delta_y_last = y_last[2:end] - y_last[1:end-1]
        comp_max_elong_sq = delta_y_last.^2
        new_max_elong_sq = max.(value.(max_elong_sq), comp_max_elong_sq)
        set_value.(max_elong_sq, new_max_elong_sq)
    
        # check for broken links
        new_broken = new_max_elong_sq .> break_dist^2
        set_value.(broken, new_broken)

        # new -> old
        # note: delta_y is the variable not y
        set_value.(prev_delta_y, value.(delta_y_last))
        prev_L = L
        L = start_L + dirichlet_delta((step-1)/fps)

        # update rod length
        set_normalized_rhs(dirichlet_right, L)

        # update start value
        delta_L = L - prev_L
        set_start_value.(delta_y, delta_y_last .+ delta_L/num_bonds)

        optimize!(minmove)

        push!(steps, (value.(y), value.(broken) .== 1.))
    end
    
    steps
end

In [ ]:
steps = compute_line_minmoves();

In [ ]:
function setup_line_fig()  
    aspect = 6
    width = animation_width
    height = width/aspect
    max_right = start_L + max_pull
    min_left = 0.
    delta = 0.25
    max_len = max_right - min_left + 2*delta
    ax_height = max_len/aspect
    
    fig = Figure(
        resolution=(animation_scale*width, animation_scale*height),
        fontsize=animation_scale*fontsize
    )
    ax = Axis(
        fig[1, 1],
        limits=(min_left-delta, max_right+delta, -ax_height/2, ax_height/2),
        aspect=aspect,
    )
    
    hidespines!(ax)
    hidedecorations!(ax)
    
    fig, ax
end

function plot_step!(ax, step)
    y = zeros(num_atoms)
    x, broken = step
    
    for i in 1:num_bonds
        if !broken[i]
            lines!(ax, [x[i], x[i+1]], [0, 0], color=:green, linewidth=animation_scale*2)
        end
    end
    
    scatter!(x, y, color=:blue, markersize=animation_scale*6)
end

function plot_line_step(step, file_name)
    if !ispath(experiments_folder)
        mkdir(experiments_folder)
    end
    
    fig, ax = setup_line_fig()
    
    plot_step!(ax, step)
    save("$experiments_folder/$file_name.pdf", fig)
    
    fig
end

function animate_line_steps(steps, file_name)
    if !ispath(experiments_folder)
        mkdir(experiments_folder)
    end
    if !only_video
        snapshots_folder = "$experiments_folder/$file_name"
        if !ispath(snapshots_folder)
            mkdir(snapshots_folder)
        end
    end
    
    fig, ax = setup_line_fig()
    
    frame = 1
    record(fig, "experiments/$file_name.mp4", steps; framerate = animation_fps) do (step)
        empty!(ax)
        plot_step!(ax, step)
        if !only_video
            save("$snapshots_folder/$(file_name)_$frame.png", fig)
        end
        frame += 1
    end
end

In [ ]:
plot_line_step(steps[end], "line_step")

In [ ]:
animate_line_steps(steps[1:2:end], "line")